<a href="https://colab.research.google.com/github/bzhao10/cs598_dlh_final_project/blob/main/preprocessing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CS 598 Final Project

In [ ]:
!pip install -U strsimpy
!pip3 install simplejson

In [ ]:
import pandas as pd
import time
import numpy as np
import string
import nltk
from nltk import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
nltk.download('punkt')
notes=pd.read_csv(r'C:\Users\pengz3\Downloads\noteevents_icd_codes.csv')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pengz3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
''' Preprocessed Data '''
'''
    (1) Punctuation characters, except for the apostrophe, are converted to whitespace
    (2) Digits are replaced by the character ’d’
    (3) All characters are converted to lowercase 
'''
#pre_process data content
notes.TEXT = notes.TEXT.str.replace('\n', '')  # remove newline
notes.TEXT = notes.TEXT.str.replace('\r', '')  # carriage returns
table = str.maketrans('!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', ' '*len('!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'))
notes.TEXT = [w.translate(table) for w in notes.TEXT]
table2 = str.maketrans('','','1234567890')
notes.TEXT = [w.translate(table2) for w in notes.TEXT]
notes.TEXT = notes.TEXT.str.lower()

In [ ]:
'''
  (4) The report is split at whitespace characters
'''
#tokenize data
tk = WhitespaceTokenizer()
notes['tokens'] = notes['TEXT']
corpus = []
count = 0
for i in range(len(notes)):
  count += 1
  if count%10000 == 0:
    print('processed count: ', count)
  note_tokenized = word_tokenize(notes['TEXT'][i])
  notes.at[i, 'tokens'] = note_tokenized
  corpus += note_tokenized
print(notes['tokens'])

processed count:  10000
processed count:  20000
processed count:  30000
processed count:  40000
processed count:  50000
processed count:  60000
processed count:  70000
processed count:  80000
processed count:  90000
processed count:  100000
processed count:  110000
processed count:  120000
processed count:  130000
processed count:  140000
processed count:  150000
processed count:  160000
processed count:  170000
processed count:  180000
processed count:  190000
processed count:  200000
processed count:  210000
processed count:  220000
processed count:  230000
processed count:  240000
0         [acute, pain, assessment, pt, has, generalized...
1         [subjective, patient, did, not, eat, breakfast...
2         [acute, pain, assessment, pt, c, o, back, and,...
3         [acute, pain, assessment, pt, has, generalized...
4         [patient, pod, c, c, anterior, fusion, pod, c,...
                                ...                        
240562    [pain, control, acute, pain, chronic, p

In [ ]:
'''
count distinct words amount
'''
from collections import Counter
items = Counter(corpus).keys()
print(len(items))

157674


In [ ]:
# with open("/content/drive/MyDrive/DLH_final_project_dataset/corpus_line.txt", "w") as opfile:
#     opfile.write("\n".join(corpus))

# with open("/content/drive/MyDrive/DLH_final_project_dataset/corpus_space.txt", "w") as opfile:
#     opfile.write(" ".join(corpus))

In [ ]:
'''
  (5) Tokens that occur less than 5 times in the whole corpus are eliminated. 
  (6) These eliminated tokens are then mapped to their most similar token of the ones which were kept. 
'''
from strsimpy.levenshtein import Levenshtein
import timeit
levenshtein = Levenshtein()
words_counts =pd.value_counts(corpus)
print('words_counts:', words_counts)
rare_words = words_counts[words_counts<5].index.tolist()
common_words = words_counts[words_counts>=5].index.tolist()
print(len(rare_words))
print(len(common_words))

words_counts: to             1674085
and            1496340
with           1028895
of              982619
on              792101
                ...   
diurseisk            1
reviewpa             1
dpressions           1
rewsponds            1
epidgastric          1
Length: 157674, dtype: int64
112525
45149


In [ ]:
''' 
 (7) keep only reports which have more than 9 and less than 2200 tokens.
'''
print('report_size: ', len(notes))
notes = notes.reset_index()
for i in range(len(notes)): 
  token_list = notes['tokens'][i]
  if len(token_list) < 9 or len(token_list) >2200:
    notes = notes.drop(i)
    continue
print('report_size_after: ', len(notes)) #before 240567 after 239205

report_size:  240567
report_size_after:  239205


In [ ]:
'''
  (8) The most similar token is defned as the one which minimizes the Levenshtein distance string metric.
'''
import json
with open(r'C:\Users\pengz3\Downloads\rare_common_dict_new.json') as json_file:
    rare_common_words_mapping= json.load(json_file)
new_corpus = []
count = 0 
notes = notes.reset_index()
for j in range(len(notes)):
  count +=1
  if count %10000 == 0:
    print('count num: ', count)
  token_list = notes['tokens'][j]
  for i, item in enumerate(token_list):
    if item in rare_common_words_mapping:
      token_list[i] = rare_common_words_mapping.get(item)
  new_corpus += token_list

count num:  10000
count num:  20000
count num:  30000
count num:  40000
count num:  50000
count num:  60000
count num:  70000
count num:  80000
count num:  90000
count num:  100000
count num:  110000
count num:  120000
count num:  130000
count num:  140000
count num:  150000
count num:  160000
count num:  170000
count num:  180000
count num:  190000
count num:  200000
count num:  210000
count num:  220000
count num:  230000


In [ ]:
'''
count distinct words amount
'''
from collections import Counter
items = Counter(new_corpus).keys()
print(len(items)) #before: 157782 after:45340

45340


In [ ]:
#Add preprocessed tokens back to TEXT
count = 0 
for j in range(len(notes)):
  count +=1
  if count %10000 == 0:
    print('count num: ', count)
  token_list = notes['tokens'][j]
  notes['TEXT'][j] = ' '.join(token_list)

C:\Users\pengz3\AppData\Local\Temp/ipykernel_18820/1284499183.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes['TEXT'][j] = ' '.join(token_list)


count num:  10000
count num:  20000
count num:  30000
count num:  40000
count num:  50000
count num:  60000
count num:  70000
count num:  80000
count num:  90000
count num:  100000
count num:  110000
count num:  120000
count num:  130000
count num:  140000
count num:  150000
count num:  160000
count num:  170000
count num:  180000
count num:  190000
count num:  200000
count num:  210000
count num:  220000
count num:  230000


In [ ]:
#generate unique reg_codes_list
reg_icd_codes_str_list= notes['reg_icd_codes'].tolist()
reg_icd_codes_all = []
for item in reg_icd_codes_str_list:
  li = list(item.split(","))
  reg_icd_codes_all += li
reg_unique_list = np.array(reg_icd_codes_all)
reg_unique_list = np.unique(reg_unique_list)
print('reg_code_size: ',len(reg_unique_list))

#generate unique rolled_up_list:
rolled_up_icd_codes_str_list= notes['rolled_up_icd_codes'].tolist()
rolled_up_icd_codes_all = []
for item in rolled_up_icd_codes_str_list:
  li = list(item.split(","))
  rolled_up_icd_codes_all += li
rolled_up_unique_list = np.array(rolled_up_icd_codes_all)
rolled_up_unique_list = np.unique(rolled_up_unique_list)
print('rolled_up_code_size: ', len(rolled_up_unique_list))

#generate unique category_list:
category_str_list= notes['CATEGORY'].tolist()
category_all = []
for item in category_str_list:
  li = list(item.split(","))
  category_all += li
category_unique_list = np.array(category_all)
category_unique_list = np.unique(category_unique_list)
print('category_size: ',len(category_unique_list))


reg_code_size:  3485
rolled_up_code_size:  737
category_size:  11


In [ ]:
reg_notes = notes[['TEXT']]
for item in reg_unique_list:
  reg_notes['reg_code_' + item.rstrip()] = np.where(notes['reg_icd_codes'].str.contains(pat = item), 1, 0)
for item in category_unique_list:
  reg_notes['category_' + item.rstrip()] = np.where(notes['CATEGORY'] == item, 1, 0)
print(reg_notes)

C:\Users\pengz3\AppData\Local\Temp/ipykernel_18820/334706856.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reg_notes['reg_code_' + item.rstrip()] = np.where(notes['reg_icd_codes'].str.contains(pat = item), 1, 0)
C:\Users\pengz3\AppData\Local\Temp/ipykernel_18820/334706856.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  reg_notes['reg_code_' + item.rstrip()] = np.where(notes['reg_icd_codes'].str.contains(pat = item), 1, 0)


                                                     TEXT  reg_code_0030  \
0       acute pain assessment pt has generalized as we...              0   
1       subjective patient did not eat breakfast today...              0   
2       acute pain assessment pt c o back and neck pai...              0   
3       acute pain assessment pt has generalized as we...              0   
4       patient pod c c anterior fusion pod c laminect...              0   
...                                                   ...            ...   
239200  pain control acute pain chronic pain assessmen...              0   
239201  pain control acute pain chronic pain assessmen...              0   
239202  hypertension benign assessment pt hypertensive...              0   
239203  ccu nursing admit note cont arrived to ccu at ...              0   
239204  ccu nursing progress notes my pressure is lowe...              0   

        reg_code_0048  reg_code_00581  reg_code_0074  reg_code_00845  \
0              

C:\Users\pengz3\AppData\Local\Temp/ipykernel_18820/334706856.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  reg_notes['category_' + item.rstrip()] = np.where(notes['CATEGORY'] == item, 1, 0)


In [ ]:
rolled_up_notes = notes[['TEXT']]
for item in rolled_up_unique_list:
  rolled_up_notes['rolled_up_code_' + item.rstrip()] = np.where(notes['rolled_up_icd_codes'].str.contains(pat = item), 1, 0)
for item in category_unique_list:
  rolled_up_notes['category_' + item.rstrip()] = np.where(notes['CATEGORY'] == item , 1, 0)
print(rolled_up_notes)

C:\Users\pengz3\AppData\Local\Temp/ipykernel_18820/1487981942.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rolled_up_notes['rolled_up_code_' + item.rstrip()] = np.where(notes['rolled_up_icd_codes'].str.contains(pat = item), 1, 0)
C:\Users\pengz3\AppData\Local\Temp/ipykernel_18820/1487981942.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rolled_up_notes['rolled_up_code_' + item.rstrip()] = np.where(notes['rolled_up_icd_codes'].str.contains(pat = item), 1, 0)


                                                     TEXT  rolled_up_code_003  \
0       acute pain assessment pt has generalized as we...                   0   
1       subjective patient did not eat breakfast today...                   0   
2       acute pain assessment pt c o back and neck pai...                   0   
3       acute pain assessment pt has generalized as we...                   0   
4       patient pod c c anterior fusion pod c laminect...                   0   
...                                                   ...                 ...   
239200  pain control acute pain chronic pain assessmen...                   0   
239201  pain control acute pain chronic pain assessmen...                   0   
239202  hypertension benign assessment pt hypertensive...                   0   
239203  ccu nursing admit note cont arrived to ccu at ...                   0   
239204  ccu nursing progress notes my pressure is lowe...                   0   

        rolled_up_code_004 

C:\Users\pengz3\AppData\Local\Temp/ipykernel_18820/1487981942.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rolled_up_notes['category_' + item.rstrip()] = np.where(notes['CATEGORY'] == item , 1, 0)


In [ ]:
column_dict_10000 = {
    'category_Nursing/other':4505,
    'category_Nursing':2886,
    'category_Physician':1838,
    'category_Respiratory':428,
    'category_Nutrition':125,
    'category_General':110,
    'category_Rehab Services':66,
    'category_Social Work':26,
    'category_Case Management':14,
    'category_Consult':1,
    'category_Pharmacy':1,
}

In [ ]:
column_dict_20000 = {
    'category_Nursing/other':9009,
    'category_Nursing':5771,
    'category_Physician':3676,
    'category_Respiratory':856,
    'category_Nutrition':250,
    'category_General':220,
    'category_Rehab Services':132,
    'category_Social Work':53,
    'category_Case Management':28,
    'category_Consult':3,
    'category_Pharmacy':2,
}

In [ ]:
#save 10000 or 20000 rolled up icd records
preprocessed_rolled_up_notes = rolled_up_notes.loc[rolled_up_notes['category_Nursing/other']==1].head(column_dict_20000['category_Nursing/other'])
for key in column_dict_20000:
    if key != 'category_Nursing/other':
        preprocessed_rolled_up_notes = preprocessed_rolled_up_notes.append(rolled_up_notes.loc[rolled_up_notes[key]==1].head(column_dict_20000[key]))
print(preprocessed_rolled_up_notes)

                                                    TEXT  rolled_up_code_003  \
41     ccu nursing progress notepls see transfer summ...                   0   
42     nursing progress notes month only i have some ...                   0   
43     respiratory care pt extubated to a aerosol avr...                   0   
44     respiratory care patient remains on cpap psv v...                   0   
45     repeat k hct due post blood transfusion lactol...                   0   
...                                                  ...                 ...   
4889   title cardiology follow up events patient note...                   0   
5500   consult requested by ccu team chief complaint ...                   0   
5517   consult requested by ccu team chief complaint ...                   0   
3652   pharmacy note digibind dose assessment pharmac...                   0   
24951  pharmacy note sedation assessment mr known las...                   0   

       rolled_up_code_004  rolled_up_co

In [ ]:
preprocessed_rolled_up_notes.loc[:,'category_Nursing/other']=preprocessed_rolled_up_notes.loc[:,'category_Nursing/other']+preprocessed_rolled_up_notes.loc[:,'category_Nursing']
preprocessed_rolled_up_notes.drop('category_Nursing', axis=1, inplace=True)

In [ ]:
preprocessed_rolled_up_notes=preprocessed_rolled_up_notes.loc[:,(preprocessed_rolled_up_notes!=0).any(axis=0)]

In [ ]:
print(preprocessed_rolled_up_notes)

                                                    TEXT  rolled_up_code_008  \
41     ccu nursing progress notepls see transfer summ...                   0   
42     nursing progress notes month only i have some ...                   0   
43     respiratory care pt extubated to a aerosol avr...                   0   
44     respiratory care patient remains on cpap psv v...                   0   
45     repeat k hct due post blood transfusion lactol...                   0   
...                                                  ...                 ...   
4889   title cardiology follow up events patient note...                   0   
5500   consult requested by ccu team chief complaint ...                   0   
5517   consult requested by ccu team chief complaint ...                   0   
3652   pharmacy note digibind dose assessment pharmac...                   0   
24951  pharmacy note sedation assessment mr known las...                   0   

       rolled_up_code_009  rolled_up_co

In [ ]:
preprocessed_rolled_up_notes.to_csv(r'C:\Users\pengz3\Downloads\preprocessed_rolled_up_icd_codes_20000.csv')

In [ ]:
#save 10000 or 20000 regular icd records
preprocessed_reg_notes = reg_notes.loc[reg_notes['category_Nursing/other']==1].head(column_dict_20000['category_Nursing/other'])
for key in column_dict_20000:
    if key != 'category_Nursing/other':
        preprocessed_reg_notes = preprocessed_reg_notes.append(reg_notes.loc[reg_notes[key]==1].head(column_dict_20000[key]))
preprocessed_reg_notes=preprocessed_reg_notes.loc[:,(preprocessed_reg_notes!=0).any(axis=0)]
print(preprocessed_reg_notes.shape)
print(preprocessed_reg_notes)

(20000, 1545)
                                                    TEXT  reg_code_00845  \
41     ccu nursing progress notepls see transfer summ...               0   
42     nursing progress notes month only i have some ...               0   
43     respiratory care pt extubated to a aerosol avr...               0   
44     respiratory care patient remains on cpap psv v...               0   
45     repeat k hct due post blood transfusion lactol...               0   
...                                                  ...             ...   
4889   title cardiology follow up events patient note...               0   
5500   consult requested by ccu team chief complaint ...               0   
5517   consult requested by ccu team chief complaint ...               0   
3652   pharmacy note digibind dose assessment pharmac...               0   
24951  pharmacy note sedation assessment mr known las...               0   

       reg_code_00869  reg_code_0088  reg_code_0090  reg_code_0091  \
41 

In [ ]:
preprocessed_reg_notes.loc[:,'category_Nursing/other']=preprocessed_reg_notes.loc[:,'category_Nursing/other']+preprocessed_reg_notes.loc[:,'category_Nursing']
preprocessed_reg_notes.drop('category_Nursing', axis=1, inplace=True)
preprocessed_reg_notes=preprocessed_reg_notes.loc[:,(preprocessed_reg_notes!=0).any(axis=0)]

In [ ]:
print(preprocessed_reg_notes)

                                                    TEXT  reg_code_00845  \
41     ccu nursing progress notepls see transfer summ...               0   
42     nursing progress notes month only i have some ...               0   
43     respiratory care pt extubated to a aerosol avr...               0   
44     respiratory care patient remains on cpap psv v...               0   
45     repeat k hct due post blood transfusion lactol...               0   
...                                                  ...             ...   
4889   title cardiology follow up events patient note...               0   
5500   consult requested by ccu team chief complaint ...               0   
5517   consult requested by ccu team chief complaint ...               0   
3652   pharmacy note digibind dose assessment pharmac...               0   
24951  pharmacy note sedation assessment mr known las...               0   

       reg_code_00869  reg_code_0088  reg_code_0090  reg_code_0091  \
41               

In [ ]:
preprocessed_reg_notes.to_csv(r'C:\Users\pengz3\Downloads\preprocessed_reg_icd_codes_20000.csv')

In [ ]:
from statistics import mean
tokens_count = []
for item in notes['tokens']:
    item_set = set(item)
    tokens_count.append(len(item_set))
print(mean(tokens_count))